In [1]:
require 'torch'
require 'optim'
require 'os'
require 'optim'
require 'xlua'
-- require 'cunn'
-- require 'cudnn' -- faster convolutions

--[[
--  Hint:  Plot as much as you can.  
--  Look into torch wiki for packages that can help you plot.
--]]

local tnt = require 'torchnet'
local image = require 'image'
local optParser = require 'opts'
local opt = optParser.parse(arg)

local WIDTH, HEIGHT = 32, 32
local DATA_PATH = (opt.data ~= '' and opt.data or './data/')

torch.setdefaulttensortype('torch.DoubleTensor')

-- torch.setnumthreads(1)
torch.manualSeed(opt.manualSeed)
-- cutorch.manualSeedAll(opt.manualSeed)

function resize(img)
    return image.scale(img, WIDTH,HEIGHT)
end

function crop(img,x1,x2,y1,y2)
    return image.crop(img, x1, y1, x2, y2)
end

function convert(img)
    return image.rgb2lab(img)

function removemean(img)
    return img - torch.mean(img)
--[[
-- Hint:  Should we add some more transforms? shifting, scaling?
-- Should all images be of size 32x32?  Are we losing 
-- information by resizing bigger images to a smaller size?
--]]
function transformInput(img)
    f = tnt.transform.compose{
        [1] = resize,
        [2] = colorscale,
        [3] = removemean
    }
    return f(img)
end

function rotatejitter(img)
    local rand_angle = (torch.randn(1)*15*3.14/180)[1]
    return image.rotate(img, rand_angle)
end

function translatejitter(img)
    local rand_position_x = (torch.randn(1)*2)[1]
    local rand_position_y = (torch.randn(1)*2)[1]
    return image.translate(img, rand_position_x, rand_position_y)   
end

function getJittered(img)
    f = tnt.transform.compose{
        [1] = rotatejitter,
        [2] = translatejitter
    }
    return f(img)
end

function getTrainSample(dataset, idx, orig)
    r = dataset[idx]
    classId, track, file = r[9], r[1], r[2]
    x1, y1, x2, y2 = r[5], r[6], r[7], r[8]
    file = string.format("%05d/%05d_%05d.ppm", classId, track, file)
    cropped = crop(image.load(DATA_PATH .. '/train_images/'..file), x1, y1, x2, y2)
    return transformInput(cropped)
end

function getTrainLabel(dataset, idx)
    return torch.LongTensor{dataset[idx][9] + 1}
end

function getTestSample(dataset, idx)
    r = dataset[idx]
    file = DATA_PATH .. "/test_images/" .. string.format("%05d.ppm", r[1])
    return transformInput(image.load(file))
end

function getIterator(dataset)
    --[[
    -- Hint:  Use ParallelIterator for using multiple CPU cores
    --]]
    return tnt.DatasetIterator{
        dataset = tnt.BatchDataset{
            batchsize = opt.batchsize,
            dataset = dataset
        }
    }
end

local trainData = torch.load(DATA_PATH..'train.t7')
local testData = torch.load(DATA_PATH..'test.t7')

trainDataset = tnt.SplitDataset{
    partitions = {train=0.9, val=0.1},
    initialpartition = 'train',
    --[[
    --  Hint:  Use a resampling strategy that keeps the 
    --  class distribution even during initial training epochs 
    --  and then slowly converges to the actual distribution 
    --  in later stages of training.
    --]]
    dataset = tnt.ShuffleDataset{
        dataset = tnt.ListDataset{
            list = torch.range(1, trainData:size(1)*5):long(),
            load = function(idx)
                preimage =  getTrainSample(trainData, idx/5),
                return {
                    input = getJittered(preimage),
                    target = getTrainLabel(trainData, idx/5)
                }
            end
        }
    }

}

[string "require 'torch'..."]:39: 'end' expected (to close 'function' at line 36) near 'function': 